# AIエージェントの構築

## 1.準備

In [ ]:
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Annotated, Literal, TypedDict
import uuid

# LangChain / OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, BaseMessage

# LangGraph Core
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2.ツールの定義

In [ ]:
# ツールの初期化
tavily_search = TavilySearch(
    max_results=2,                 # 取得する検索結果の数
    search_depth="basic",          # "basic" (高速) か "advanced" (高品質)
    include_answer=False,          # Tavilyが生成した短い回答を含めない
    include_raw_content=False,     # HTMLの生コンテンツを含めるか（トークン消費増に注意）
    include_images=False,          # 画像URLを含めるか
    # include_domains=["go.jp"],   # 特定のドメインのみ検索する場合
    # exclude_domains=["wikipedia.org"] # 特定のドメインを除外する場合
)

# Tavilyの検索結果だけを、LLMが読みやすい形に整形する関数
def format_tavily_results(tavily_response: dict) -> str:
    results = tavily_response.get("results", [])
    if not results:
        return "（検索結果なし）"

    lines = []
    for i, r in enumerate(results, 1):
        title = r.get("title", "")
        content = r.get("content", "")
        url = r.get("url", "")
        lines.append(f"[{i}] {title}\n{content}\nsource: {url}")
    return "\n\n".join(lines)

# TavilySearchの生レスポンス(dict/JSON)を整形して返すラッパーTool
@tool
def tavily_search_formatted(query: str) -> str:
    """Web検索（Tavily）。上位結果を整形して返す。"""
    tavily_response = tavily_search.invoke({"query": query})
    return format_tavily_results(tavily_response)

tools = [tavily_search_formatted]

## 3.ステート、モデルの定義

In [ ]:
class State(TypedDict):
    # add_messagesを使うことで履歴を追記型で管理
    research_messages: Annotated[list[BaseMessage], add_messages]
    analysis_messages: Annotated[list[BaseMessage], add_messages]
    # カウンターなどは上書きでOK
    loop_count: int

# モデルの初期化
model = ChatOpenAI(model=MODEL_NAME)
model_with_tools = model.bind_tools(tools)

## 4.ノードの定義

### デバッグ用関数

In [ ]:
# デバッグ用関数
DEBUG_MODE = True  # ここをTrueにすると詳細ログが出ます

def print_debug(title, content):
    if DEBUG_MODE:
        print(f"\n🐛 [DEBUG] {title}:\n{content}\n" + "-"*40)

### リサーチエージェント

In [ ]:
MAX_TOOL_LOOPS = 3

research_prompt_text = """
あなたは事業リサーチ担当です。ユーザーのテーマについて、市場規模、主要プレイヤー、技術課題などをWeb検索で調査してください。必要に応じて最適なツールを使用してください。
ツール呼び出しを行った後に回答する場合（ツール結果にURLが含まれる場合のみ）は以下を行ってください。
・回答の末尾に参照したsource番号（例：[1][2]）を付ける
・最後にsource番号とurlを箇条書きで列挙する
"""

research_prompt = ChatPromptTemplate.from_messages([
    ("system", research_prompt_text),
    MessagesPlaceholder(variable_name="research_messages")
])
research_chain = research_prompt | model_with_tools

def research_agent(state: State) -> Command[Literal["tools", "summary_agent"]]:
    print("🔍 リサーチエージェント: 思考中...")

    response = research_chain.invoke({"research_messages": state["research_messages"]})

    update = {"research_messages": [response]}
    current_count = state.get("loop_count", 0)

    # ツール呼び出し判定
    if response.tool_calls:
        # デバッグ：複数 tool_call に対応して表示
        tool_names = [tc.get("name") for tc in response.tool_calls]
        print(f"👉 ツール呼び出し: {tool_names}")
        if DEBUG_MODE:
            for i, tc in enumerate(response.tool_calls, 1):
                print_debug(f"Tool Call #{i} ({tc.get('name')})", tc.get("args"))

        if current_count < MAX_TOOL_LOOPS:
            return Command(update=update, goto="tools")
        else:
            # 上限到達でも、直前のAIMessageは必ず履歴に残す
            print("⚠️ ループ上限のため、サマリーへ強制移行します。")
            return Command(update=update, goto="summary_agent")

    print("✅ リサーチ完了。サマリーへ移行します。")
    return Command(update=update, goto="summary_agent")

### ツールノード

In [ ]:
# ToolNodeの使用（並列ツール実行も自動対応）
tool_node = ToolNode(tools, messages_key="research_messages")

# ツールノード（ラッパー）
def research_tool_node(state: State) -> Command[Literal["research_agent"]]:
    print("🛠️ ツール実行中...")

    # 標準のToolNodeを実行
    result = tool_node.invoke({"research_messages": state["research_messages"]})
    
    # デバッグ: ツールの実行結果を表示
    # ToolMessageのcontentを表示
    last_message = result["research_messages"][-1]
    tool_text = last_message.content
    tool_text = tool_text if isinstance(tool_text, str) else str(tool_text)
    print_debug("Tool Output", tool_text[:300] + "... (省略)")

    return Command(
        update={
            "research_messages": result["research_messages"],
            "loop_count": state.get("loop_count", 0) + 1
        },
        goto="research_agent"
    )

### サマリーエージェント

In [ ]:
# サマリーエージェント
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは優秀な書記です。以下の「調査ログ」の内容を要約し、次の市場分析チームが使える詳細なレポートを作成してください。調査ログに含まれる source: URL を参照し、断定的な事実には可能な限り [n] を付け、末尾に参照一覧を付ける。存在しない出典は作らない。"),
    ("human", "以下が調査ログです:"),
    MessagesPlaceholder(variable_name="research_messages"),
    ("human", "上記を元に、市場分析のための基礎レポートを作成してください。")
])
summary_chain = summary_prompt | model

def summary_agent(state: State) -> dict:
    print("📝 サマリーエージェント: 調査結果を分析チームへ共有します...")
    response = summary_chain.invoke({"research_messages": state["research_messages"]})
    return {"analysis_messages": [response]}

### 市場分析エージェント

In [ ]:
# 市場分析エージェント
market_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは市場分析のプロです。レポートを元にSWOT分析を行ってください。"),
    MessagesPlaceholder(variable_name="analysis_messages")
])
market_chain = market_prompt | model

def market_agent(state: State) -> dict:
    print("📈 市場分析エージェント: 分析中...")
    response = market_chain.invoke({"analysis_messages": state["analysis_messages"]})
    return {"analysis_messages": [response]}

### 技術分析エージェント

In [ ]:
technical_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは技術のCTOです。市場分析を踏まえ、技術的課題と実現性を指摘してください。"),
    MessagesPlaceholder(variable_name="analysis_messages")
])
technical_chain = technical_prompt | model

# 技術分析エージェント
def technical_agent(state: State) -> dict:
    print("💻 技術分析エージェント: 分析中...")
    response = technical_chain.invoke({"analysis_messages": state["analysis_messages"]})
    return {"analysis_messages": [response]}

### Human-in-the-loop (HITL) ノード

In [ ]:
# HITLノード
def human_approval_node(state: State) -> Command[Literal["market_agent", "report_agent", "__end__"]]:
    """
    - interrupt() の引数（payload）は JSON-serializable な dict を推奨
    - resume された値が interrupt() の戻り値として入ってくる
    """
    # UI/クライアント側に提示したい情報を payload に入れる（JSON-serializable）
    payload = {
        "kind": "approval_request",
        "question": "ここまでの議論を承認してレポートを作成しますか？",
        "options": ["y", "n", "retry"],
        "theme": (
            state["research_messages"][0].content
            if state.get("research_messages") else ""
        ),
        "analysis_preview": [
            {
                "type": type(m).__name__,
                "content": (m.content[:500] + "…") if isinstance(m.content, str) and len(m.content) > 500 else m.content
            }
            for m in state.get("analysis_messages", [])
        ],
    }

    # interrupt：ここで停止し、resume 後は「戻り値」にユーザー入力が入る
    user_decision = interrupt(payload)

    # 再開後のロジック
    if isinstance(user_decision, str):
        user_decision = user_decision.strip().lower()

    if user_decision == "y":
        print("✅ 承認されました。最終レポートを作成します。")
        return Command(goto="report_agent")
    elif user_decision == "retry":
        print("🔄 議論を再開します（市場分析へ戻る）")
        return Command(goto="market_agent")
    else:
        print("🛑 終了します（レポートは作成しません）。")
        return Command(goto=END)

### レポート作成エージェント

In [ ]:
report_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    これまでの議論をすべて統合し、投資家向けの具体的な事業プランを作成してください。
    文末での質問や提案は禁止です。「以上」で終わらせてください。
    """),
    MessagesPlaceholder(variable_name="analysis_messages")
])
report_chain = report_prompt | model

# レポート作成エージェント
def report_agent(state: State) -> dict:
    print("📄 レポート作成エージェント: 最終成果物を作成中...")
    response = report_chain.invoke({"analysis_messages": state["analysis_messages"]})
    return {"analysis_messages": [response]}

## 5. グラフ構築

In [ ]:
builder = StateGraph(State)

# ノード追加
builder.add_node("research_agent", research_agent)
builder.add_node("tools", research_tool_node)
builder.add_node("summary_agent", summary_agent)
builder.add_node("market_agent", market_agent)
builder.add_node("technical_agent", technical_agent)
builder.add_node("human_approval", human_approval_node)
builder.add_node("report_agent", report_agent)

# エッジ定義 (Commandを使っているため、明示的なadd_edgeはフローの固定部分だけでOK)
builder.add_edge(START, "research_agent")
builder.add_edge("summary_agent", "market_agent")
builder.add_edge("market_agent", "technical_agent")
builder.add_edge("technical_agent", "human_approval")
builder.add_edge("report_agent", END)

# コンパイル（interrupt/resume を使うために checkpointer は必須）
memory = InMemorySaver()
app = builder.compile(checkpointer=memory)

# グラフの可視化
display(Image(app.get_graph().draw_mermaid_png()))

## 6.メイン処理

In [ ]:
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

# テーマ入力
initial_theme = input("テーマを入力してください:") or "宇宙ゴミの回収事業"
print(f"テーマ：{initial_theme}")

# 初期ステート
initial_state = {
    "research_messages": [HumanMessage(content=f"テーマ: {initial_theme}")],
    "loop_count": 0,
    "analysis_messages": [],
}

print("🚀 エージェントを実行します...")

# まずは最初の実行（完了するか、interrupt で止まるまで進む）
result = app.invoke(initial_state, config=config)

while True:
    interrupts = result.get("__interrupt__", None)

    # 1) interrupt 判定（停止中）
    if interrupts:
        # list/tuple/単体どれでも受ける
        first = interrupts[0] if isinstance(interrupts, (list, tuple)) else interrupts
        # Interruptオブジェクトなら .value、そうでなければそのまま
        interrupt_payload = getattr(first, "value", first)

        print("\n" + "=" * 20 + " 📝 ここまでの議論のまとめ " + "=" * 20)

        # payload 側に analysis_preview を入れているので、それを優先して表示
        if isinstance(interrupt_payload, dict) and "analysis_preview" in interrupt_payload:
            previews = interrupt_payload["analysis_preview"]
            if previews:
                for item in previews:
                    print(f"\n--- 🗣️ {item.get('type', 'Message')} ---\n{item.get('content', '')}\n")
            else:
                print("（分析データはありません）")
        else:
            # 互換：payloadに含まれない場合は state から表示
            current_analysis = result.get("analysis_messages", [])
            if current_analysis:
                for msg in current_analysis:
                    content = msg.content if hasattr(msg, "content") else str(msg)
                    content = content if isinstance(content, str) else str(content)
                    print(f"\n--- 🗣️ {type(msg).__name__} ---\n{content[:500]}...\n(以下略)\n")
            else:
                print("（分析データはありません）")

        print("=" * 66 + "\n")

        # 2) 承認プロンプト（payloadの質問文を使う）
        if isinstance(interrupt_payload, dict):
            question = interrupt_payload.get("question", "承認しますか？ (y/n/retry)")
            options = interrupt_payload.get("options", ["y", "n", "retry"])
        else:
            question = "承認しますか？ (y/n/retry)"
            options = ["y", "n", "retry"]

        print(f"✋ 【承認待ち】{question}")
        print(f"🤖 System Message: 入力してください {options}")

        user_input = input("User Input (y/n/retry) > ").strip().lower()
        print(f"🔄 Resuming with input: '{user_input}'...")

        # 3) 再開（resume 値が interrupt() の戻り値に入る）
        result = app.invoke(Command(resume=user_input), config=config)
        continue

    # 2) 完了（interrupt なし）
    print("\n✨ 全工程が完了しました。")
    analysis_messages = result.get("analysis_messages", [])
    if analysis_messages:
        last = analysis_messages[-1]
        last_content = last.content if hasattr(last, "content") else str(last)
        print("=" * 60)
        print(last_content)
        print("=" * 60)
    break